In [5]:
import numpy as np
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt
from scipy.integrate import quad, fixed_quad, tplquad, dblquad

En un cirtcuito con un voltaje de ε(t)
 y una inductancia de L
, la primera ley de Kirchhoff nos da la siguiente relación

ε(t)=Ldidt+Ri
donde R
, es la resistencia del ciruito, I
 es la corriente. Suponga que medimos la corriente con varios valores de t
 y obtenemos:

t = np.array([1.00, 1.01, 1.02, 1.03, 1.04])  # tiempo
i = np.array([3.10, 3.12, 3.14, 3.18, 3.24])  # corriente
donde t
 se mide en segundos, I
 se da en Amperios, la inductancia L=0.98
 H y la resistencia es de 0.142 Ω
. Aproxime el votaje ε(t)
 en los valores t=1.00, 1.01, 1.02, 1.03, 1.04 , para ello realice lo siguiente:

Construya un data frame de pandas con los valores de tiempo y corriente
Emplee el metodo de diferencias finitas y de tres puntos para calcular la derivada numérica.
Ahora, encuentre un polinomio de interpolación usando alguno de los métodos vistos y calcule la derivada 'exacta'. Compare sus resultados.

In [ ]:
t = np.array([1.00, 1.01, 1.02, 1.03, 1.04])
i = np.array([3.10, 3.12, 3.14, 3.18, 3.24])


L = 0.98
R = 0.142
h = t[1] - t[0]


di_dt_finita = np.zeros_like(i)


di_dt_finita[0] = (-3*i[0] + 4*i[1] - i[2]) / (2*h)
di_dt_finita[-1] = (3*i[-1] - 4*i[-2] + i[-3]) / (2*h)


for k in range(1, len(i)-1):
    di_dt_finita[k] = (i[k+1] - i[k-1]) / (2*h)


epsilon_finita = L * di_dt_finita + R * i


t_sym = sp.symbols('t')
poly = sp.interpolate(list(zip(t, i)), t_sym)
dpoly = sp.diff(poly, t_sym)


di_dt_poly = np.array([float(dpoly.subs(t_sym, val)) for val in t])
epsilon_poly = L * di_dt_poly + R * i

df = pd.DataFrame({
    't (s)': t,
    'i (A)': i,
    'di/dt (finita) (A/s)': di_dt_finita,
    'ε (finita) (V)': epsilon_finita,
    'di/dt (polinomio) (A/s)': di_dt_poly,
    'ε (polinomio) (V)': epsilon_poly,
    'Δε (poly - finite) (V)': epsilon_poly - epsilon_finita
})

print(df)

Utilizando los siguientes datos de tiempo y distancia:

t = np.array([0, 3, 5, 8, 10, 13])  # Tiempo en s
x = np.array([0, 225, 383, 623, 742, 993])  # Distancia recorrida en m
Determinar la velocidad instantanea para cada uno de los tiempos y distancias mostrados.

¿Cuál será la velocidad para t=12
?

In [ ]:
t = np.array([0, 3, 5, 8, 10, 13])
x = np.array([0, 225, 383, 623, 742, 993])


v = np.zeros_like(x, dtype=float)


v[0] = (x[1] - x[0]) / (t[1] - t[0])
v[-1] = (x[-1] - x[-2]) / (t[-1] - t[-2])

for k in range(1, len(t)-1):
    v[k] = (x[k+1] - x[k-1]) / (t[k+1] - t[k-1])


tsym = sp.Symbol('t')

poly = sp.interpolate(list(zip(t, x)), tsym)

dpoly = sp.diff(poly, tsym)


v_12 = float(dpoly.subs(tsym, 12))


df_vel = pd.DataFrame({
    't (s)': t,
    'x (m)': x,
    'v (m/s) [aprox]': v
})

df_vel.loc[len(df_vel)] = [12, None, v_12]


print(df_vel)

3.
image.png

Las estaciones de radar A
 y B
, separadas por una distancia a=500m
, rastrean un avión C
 registrando los ángulos α
 y β
 en intervalos de 1 segundo. Tres lecturas sucesivas son

data = {
    't (s)': [9, 10, 11],
    'alpha (°)': [54.80, 54.06, 53.34],
    'beta (°)': [65.59, 64.59, 63.62]
}

df = pd.DataFrame(data)
Calcular la velocidad v
 del avión y el ángulo de ascenso γ
 en t=10s
. Las coordenadas del avión pueden expresarse como:

x=atanβtanβ−tanα
y=atanαtanβtanβ−tanα

In [ ]:
a = 500

data = {
    't (s)': [9, 10, 11],
    'alpha (°)': [54.80, 54.06, 53.34],
    'beta (°)': [65.59, 64.59, 63.62]
}
df = pd.DataFrame(data)


alpha_rad = np.radians(df['alpha (°)'])
beta_rad = np.radians(df['beta (°)'])


tan_alpha = np.tan(alpha_rad)
tan_beta = np.tan(beta_rad)
denominator = tan_beta - tan_alpha

x = a * tan_beta / denominator
y = a * tan_alpha * tan_beta / denominator


dx_dt = (x[2] - x[0]) / (df['t (s)'][2] - df['t (s)'][0])
dy_dt = (y[2] - y[0]) / (df['t (s)'][2] - df['t (s)'][0])


v = np.sqrt(dx_dt**2 + dy_dt**2)
gamma_rad = np.arctan2(dy_dt, dx_dt)
gamma_deg = np.degrees(gamma_rad)


print(f"Velocidad del avión: {v:.2f} m/s")
print(f"Ángulo de ascenso: {gamma_deg:.2f}°")

Se proporcionan los siguientes datos para la velocidad de un objeto en función del tiempo:

# Datos de tiempo y velocidad
t = np.array([0, 4, 8, 12, 16, 20, 24, 28, 32, 36])  # segundos
v = np.array([0, 34.7, 61.8, 82.8, 99.2, 112.0, 121.9, 129.7, 135.7, 140.4])  # m/s
¿Qué tan lejos viaja el objeto desde t=0
 hasta 28 s?
¿Cuál es la aceleración del objeto a t=28
 s?

In [ ]:
t = np.array([0, 4, 8, 12, 16, 20, 24, 28, 32, 36])
v = np.array([0, 34.7, 61.8, 82.8, 99.2, 112.0, 121.9, 129.7, 135.7, 140.4])


t_sub = t[t <= 28]
v_sub = v[:len(t_sub)]


distance = simpson(v_sub, t_sub)


i_28 = np.where(t == 28)[0][0]

a_28 = (v[i_28 + 1] - v[i_28 - 1]) / (t[i_28 + 1] - t[i_28 - 1])


print(f"Distancia recorrida hasta t = 28 s: {distance:.2f} m")
print(f"Aceleración en t = 28 s: {a_28:.2f} m/s²")

5.
El área superficial horizontal As
 (m2
) de un lago a una profundidad particular puede calcularse a partir del volumen por diferenciación:

As(z)=−dV(z)dz
donde:

V
 = volumen (m3
)
z
 = profundidad (m
) medida desde la superficie hacia el fondo
La concentración promedio de una sustancia que varía con la profundidad c
 (g/m3
) puede calcularse por integración:

c=∫Z0c(z)As(z)dzVT
donde:

Z
 = profundidad total (m)
VT
 = volumen total del lago, dado también por:
VT=∫Z0As(z)dz
Con base en los siguientes datos, determine la concentración promedio.

# Datos proporcionados
z = np.array([0, 4, 8, 12, 16])  # profundidad en metros
V = np.array([9.8175, 5.1051, 1.9635, 0.3927, 0.0000]) * 1e6  # volumen en m³
c = np.array([10.2, 8.5, 7.4, 5.2, 4.1])  # concentración en g/m³

# Parámetros importantes
Z = z[-1]  # profundidad total (16 m)

In [3]:
# Datos proporcionados
z = np.array([0, 4, 8, 12, 16])
V = np.array([9.8175, 5.1051, 1.9635, 0.3927, 0.0000]) * 1e6
c = np.array([10.2, 8.5, 7.4, 5.2, 4.1])

# Parámetros importantes
Z = z[-1]

superficial_horizontal = lambda z: -1*(np.gradient(V))/(np.gradient(z))

volumen_total = lambda z: quad(superficial_horizontal(z),0,Z)
def concetracion_media(z):
  a = superficial_horizontal(z)
  b = lambda z: concetracion_media(z)*a
  resultado = quad(b,0,Z)/volumen_total(z)
  return resultado



La función de onda del electrón en estado 2s (n=2
, l=0
, m=0
) en unidades radiales adimensionales es:

ψ200(r,θ,φ)=142π−−√(2−r)e−r/2
La probabilidad de encontrar el electrón entre r=0
 y r=R
 está dada por:

P(R)=18∫R0r2(2−r)2e−rdr
donde las integrales angulares contribuyen un factor de 4π
.

Calcule las probabilidades P(R)
 para R∈[10,25]
 con incrementos de 1 utilizando los métodos:
Trapezoidal
Simpson
Cuadratura
Garantizando que el error sea menor a ε=10−6
Calcule la integral radial desde r=0
 hasta +∞
, verifique que la probabilidad total sea 1



In [ ]:
def integrand(r):
    return r**2 * (2 - r)**2 * np.exp(-r)

# Regla del trapecio compuesta (vectorizada)
def P_trap(R, h=1e-3):
    n = int(R / h)
    x = np.linspace(0, R, n + 1)
    fx = integrand(x)
    integral = h * (0.5 * (fx[0] + fx[-1]) + fx[1:-1].sum())
    return integral / 8

# Regla de Simpson compuesta (vectorizada)
def P_simp(R, h=1e-3):
    n = int(R / h)
    if n % 2 == 1:
        n += 1  # Simpson requiere número par de intervalos
    h = R / n
    x = np.linspace(0, R, n + 1)
    fx = integrand(x)
    integral = h / 3 * (fx[0] + fx[-1] +
                        4 * fx[1:-1:2].sum() +
                        2 * fx[2:-2:2].sum())
    return integral / 8

# Cuadratura Gauss-Legendre de orden dado
def P_gauss(R, order=64):
    nodes, weights = np.polynomial.legendre.leggauss(order)
    t = 0.5 * (nodes + 1) * R  # transformar de [-1,1] a [0,R]
    integral = 0.5 * R * np.dot(weights, integrand(t))
    return integral / 8

# Calcular probabilidades para R = 10 a 25
resultados = []
for R in range(10, 26):
    p_trap = P_trap(R)
    p_simp = P_simp(R)
    p_gauss = P_gauss(R)
    resultados.append((R, p_trap, p_simp, p_gauss))


df = pd.DataFrame(resultados, columns=["R", "Trapecio", "Simpson", "Gauss"])


print(df.to_string(index=False))


plt.figure(figsize=(10, 6))
plt.plot(df["R"], df["Trapecio"], 'o-', label="Trapecio")
plt.plot(df["R"], df["Simpson"], 's-', label="Simpson")
plt.plot(df["R"], df["Gauss"], '^-', label="Gauss-Legendre")
plt.xlabel("R")
plt.ylabel("P(R)")
plt.title("Probabilidad acumulada P(R)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Una lámina cuadrada uniforme de metal flota inmóvil en el espacio:

image.png

La lámina cuadrada tiene 10 m de lado y espesor despreciable. La componente de la fuerza gravitacional a lo largo del eje z
 sobre una masa de prueba de 1 kg ubicada a distancia z
 del centro de la lámina es:

Fz=Gσz∬L/2−L/2dxdy(x2+y2+z2)3/2
donde G=6.674×10−11m3kg−1s−2
, σ=ML2
, L=10
 m y M=10,000
 kg.

Calcular numéricamente esta integral doble.


In [ ]:
G = 6,674 * 1e-11

z = np.linspace(0,10,100)

L = 10

M = 10000

omega = M / L**2

a = - L/2
b = L/2

q = lambda y: -L/2
r = lambda y: L/2

f = lambda x,y: 1/((x**2 + y**2 + z**2)**(3/2))
integral = dblquad(f, a, b, q, r)
valor = lambda z: G * omega * z * integral

plt.plot(z,valor(z), color = "red", label = "Fuerza gravitacional" )
plt.legend()
plt.grid()
plt.show()